In [1]:
import openai
import pandas as pd
import re
from tqdm import tqdm

In [2]:
human_df = pd.read_csv("../../human_experiment/newsvendor_human_data.csv")

In [ ]:
openai.api_key = "your-api-key-here"

# Intrinsic

In [ ]:
def create_user_prompt(round_num, price, cost):
    """Create user prompt for a specific round with price and cost information."""
    return (
        f"Round {round_num}\n"
        f"- Selling Price per Unit: {price} USD\n"
        f"- Cost per Unit: {cost} USD\n"
        "Please enter the number of units you would like to order for this round."
    )

def get_system_prompt(mode, risk_attitude=None):
    """Get system prompt based on experiment mode."""
    base = (
        "You are participating in an inventory management simulation. "
        "In each round, you will decide how many units of a product to order before the selling season begins. "
        "The demand for the product is uncertain but follows a known distribution. "
        "Your objective is to **maximize your profit** over the course of the simulation. "
        "Your output should be **a integer between 0 and 300**."
    )
    
    if mode == "intrinsic":
        return base
    elif mode == "instruction" and risk_attitude == "risk-seeking":
        return base + (
            " As a risk-seeking manager, you are willing to take chances. "
            "You prefer to over-order in hopes of capturing high sales, even if it means risking unsold inventory."
        )
    elif mode == "instruction" and risk_attitude == "risk-averse":
        return base + (
            " As a risk-averse manager, you are cautious. "
            "You prefer to under-order to avoid the risk of unsold inventory, even if it means missing some potential sales."
        )
    return base

def compute_profit_sales(order, demand, price, cost):
    """Calculate profit and sales given order quantity, demand, price, and cost."""
    sales = min(order, demand)
    profit = price * sales - cost * order
    return profit, sales

def extract_order_quantity(response_content):
    """Extract integer order quantity from LLM response."""
    content = response_content.strip()
    try:
        order_quantity = int(content)
        if str(order_quantity) != content:
            raise ValueError("Output contains non-integer characters")
        return order_quantity
    except ValueError:
        digits = "".join(filter(str.isdigit, content))
        if digits:
            return int(digits)
        raise ValueError(f"LLM output is not a valid integer: {content}")

def call_llm_for_order(system_prompt, user_prompt, model="gpt-4o", temperature=1.0):
    """Make API call to get order quantity from LLM."""
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature,
        max_tokens=10,
        n=1,
        stop=None
    )
    return response.choices[0].message.content.strip()

def extract_ai_orders(text):
    """Extract (round, order) pairs from LLM output text for imitation mode."""
    pattern = r'round\s+(\d+):\s*(\d+)'
    matches = re.findall(pattern, text)
    df = pd.DataFrame(matches, columns=["round", "order"])
    df["round"] = df["round"].astype(int)
    df["order"] = df["order"].astype(int)
    return df

In [ ]:
def run_intrinsic_experiment(data):
    """Run intrinsic experiment mode where LLM acts without specific risk instructions."""
    results = []
    system_prompt = get_system_prompt("intrinsic")
    
    for index, row in tqdm(data.iterrows(), total=len(data), desc="Running intrinsic mode"):
        try:
            user_prompt = create_user_prompt(row['round'], row['price'], row['cost'])
            response_content = call_llm_for_order(system_prompt, user_prompt)
            order_quantity = extract_order_quantity(response_content)
            
            profit, sales = compute_profit_sales(order_quantity, row['demand'], row['price'], row['cost'])
            
            results.append({
                'round': row['round'],
                'participantID': row['participantID'],
                'cost': row['cost'],
                'demand': row['demand'],
                'order': order_quantity,
                'price': row['price'],
                'profit': profit,
                'sales': sales,
                'mode': 'intrinsic'
            })
            
        except Exception as e:
            print(f"Error in round {row['round']}: {e}")
            continue
    
    return pd.DataFrame(results)

intrinsic_df = run_intrinsic_experiment(human_df)


In [ ]:
intrinsic_df.to_csv("../../results/Newsvendor/intrinsic.csv", index=False)


# Instruction

In [ ]:
def run_instruction_experiment(data):
    """Run instruction experiment mode with risk-seeking and risk-averse prompts."""
    results = []
    
    for mode in ["risk-seeking", "risk-averse"]:
        system_prompt = get_system_prompt("instruction", risk_attitude=mode)
        
        for index, row in tqdm(data.iterrows(), total=len(data), desc=f"Processing {mode}"):
            try:
                user_prompt = create_user_prompt(row["round"], row["price"], row["cost"])
                response_content = call_llm_for_order(system_prompt, user_prompt)
                
                order = extract_order_quantity(response_content)
                if order < 0 or order > 300:
                    raise ValueError(f"Order quantity {order} is outside valid range 0-300")
                
                profit, sales = compute_profit_sales(order, row["demand"], row["price"], row["cost"])
                
                results.append({
                    'round': row['round'],
                    'participantID': row['participantID'],
                    'cost': row['cost'],
                    'demand': row['demand'],
                    'order': order,
                    'price': row['price'],
                    'profit': profit,
                    'sales': sales,
                    'mode': mode
                })
                
            except Exception as e:
                pass
                continue
    
    return pd.DataFrame(results)

instruction_df = run_instruction_experiment(human_df)

In [ ]:
instruction_df.to_csv("../../results/Newsvendor/instruction.csv", index=False)

# Imitation

In [ ]:
data = human_df.copy()

In [ ]:
context_num = 15

In [ ]:
# prompts
task = f"""
1. Review the participant's inventory ordering decisions from the first {context_num} rounds.
2. For rounds {context_num+1} to 30, continue their strategy by predicting order quantities that match their decision patterns.
"""

In [ ]:
data_grouped = data.groupby('participantID')
result_df = pd.DataFrame(columns=['participantID', 'round', 'order_llm', 'order_human', 'demand', 'price', 'cost'])

def extract_ai_orders(text):
    """
    Extract (round, order) pairs from LLM output text and return a DataFrame.
    Example line: 'round 16: 210'
    """
    pattern = r'round\s+(\d+):\s*(\d+)'
    matches = re.findall(pattern, text)
    df = pd.DataFrame(matches, columns=["round", "order"])
    df["round"] = df["round"].astype(int)
    df["order"] = df["order"].astype(int)
    return df

for pid, group in tqdm(data_grouped):
    group = group.sort_values('round')
    context_df_human = group[group['round'] <= context_num]
    predict_df_human = group[group['round'] > context_num]

    context_text = "\n".join([
        f"Round {r['round']}: Order={r['order']}, Demand={r['demand']}, Profit={r['profit']}"
        for _, r in context_df_human.iterrows()
    ])

    future_demand_text = "\n".join([
        f"Round {r['round']}: Demand={r['demand']}, Price={r['price']}, Cost={r['cost']}"
        for _, r in predict_df_human.iterrows()
    ])

    system_prompt = f"""
You are an autonomous agent in a 30-round inventory management experiment.

## Instructions:
In each round, you decide how many units to order before demand is realized.
Demand ranges from 1 to 300 units. Your objective is to maximize profit.
Price and cost vary by round.

## Task:
{task}

## Output Format:
Please respond using this format, one per line:

round {context_num+1}: [order]
round {context_num+2}: [order]
...
round 30: [order]
""".strip()

    user_prompt = f"""
## Participant History (Rounds 1-{context_num}):
{context_text}

## Demand & Pricing Info (Rounds {context_num+1}-30):
{future_demand_text}
""".strip()

    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=1.0,
            max_tokens=600
        )

        # Save response
        with open(f'response_log_newsvendor_{context_num}.txt', 'a') as f:
            f.write(f"\n=== Participant {pid} ===\n")
            f.write(f"System Prompt:\n{system_prompt}\n")
            f.write(f"User Prompt:\n{user_prompt}\n")
            f.write(f"Response:\n{response.choices[0].message.content}\n")
            f.write(f"Timestamp: {response.created}\n")
            f.write("=" * 80 + "\n")

        response_text = response.choices[0].message.content.strip()
        ai_orders = extract_ai_orders(response_text)
        ai_df = pd.DataFrame(ai_orders, columns=['round', 'order'])

        # Merge LLM output with metadata (demand, price, cost)
        merged = pd.merge(ai_df, predict_df_human, on='round', suffixes=('_llm', '_human'))
        merged['participantID'] = pid
        merged = merged[['participantID', 'round', 'order_human', 'order_llm', 'demand', 'price', 'cost']]

        result_df = pd.concat([result_df, merged], axis=0)

    except Exception as e:
        print(f"[ERROR] Participant {pid}: {e}")
        continue

In [ ]:
# Save result
result_df.to_csv(f"imitation.csv", index=False)